### Common measure of outcome being used for each study type?
What methods do each study types use to measure outcome?

### Study results?
From CSV file, what results can we find?

## 3.2. A deeper dive into COVID-related studies only

### Observational studies
- length of each study (from CSV)
- how is the outcome measured?

In [1]:
# from CSV, find out about length of each study and categorize into observational vs. interventional
df_studies_COVID = df_studies.loc[df_studies['study_topic_code']==1,
                                  ['condition','enrollment','id','intervention','title', 'study_topic_code']]
# df_studies_CSV.columns
# df_studies.columns

right = df_studies_CSV[['NCT Number','Outcome Measures','Phases',
                        'Start Date','Primary Completion Date','Completion Date']]

df_COVID_joined = pd.merge(df_studies_COVID, right,
                           how="inner", left_on='id',right_on='NCT Number',
                           sort=False,suffixes=("","_CSV"),
                           validate="1:1")

# Define timestamp columns and drop NA values
study_dates_names = ['Start Date','Primary Completion Date','Completion Date']
df_COVID_joined.dropna(axis=0,subset=study_dates_names,inplace=True)

# Take differences between dates to find study durations
df_COVID_joined[study_dates_names] = df_COVID_joined[study_dates_names].astype('datetime64')

df_COVID_joined['study_duration_primary'] = df_COVID_joined['Primary Completion Date'] - df_COVID_joined['Start Date']
df_COVID_joined['study_duration_total'] = df_COVID_joined['Completion Date'] - df_COVID_joined['Start Date']

# df_COVID_joined.head()

NameError: name 'df_studies' is not defined

In [2]:
# np.where(pd.DatetimeIndex(study_dates_timestamp['Start Date']).date == pd.to_datetime((2019,2020)[0]))
# pd.to_datetime((2019,2020))
# hi = (0,1,2,3)
# hi[not 0]
datetime.date(2020,1,1).year

NameError: name 'datetime' is not defined

In [ ]:
# Sort values in terms of Study start dates, oldes to newest
study_dates_timestamp = df_COVID_joined[study_dates_names].sort_values(by='Start Date')
study_dates_timestamp#.reset_index(drop=True,inplace=True)

In [ ]:
study_dates_timestamp['Start Date'].astype('datetime64[D]')

In [ ]:
def mytimeline(dataframe_orig, startyear_range, endyear = None,
               indexercol=0, increment=1, title='', ylabel='', colormap='plasma'):
    '''
    Plots a timeline for a given dataframe containing timestamp data in its columns.
    :param dataframe: pandas DataFrame containing timestamp in its columns.
    :param startyear_range: low and high limits of the range of timeline start years to be plotted; array-like of ints of size 2.
    :param endyear: higher limit of the time axis. If not provided, will be defined by data; int, default = NaN
    :param indexercol: index of column to sort dataframe rows by, and to select a portion of data to be plotted from; int
    :param increment: increment by which to increase the years plotted as xticks; default 1.
    :param title: title of resulting timeline; default ''
    :param colormap: colormap to choose colours from; each column has a different colour.
    :return: outputs the timeline plot.
    '''

    # sort dataframe
    colnames = dataframe_orig.columns.to_list()
    indexercol_name = colnames[indexercol]
    dataframe = dataframe_orig[colnames].sort_values(by=indexercol_name)
    dataframe.reset_index(drop=True, inplace=True)
    colnames.pop(indexercol)

    # Define the time range of study start dates to plot
    # To do this, I must first define first and last possible start dates
    startdate_first = datetime.date(startyear_range[0], 1, 1)
    startdate_last = datetime.date(startyear_range[1], 1, 1)

    # Then, compare that date with the indexer column
    indStart = np.where(dataframe[indexercol_name] >= datetime.datetime.combine(startdate_first, datetime.datetime.min.time()))[0][0]
    indEnd = np.where(dataframe[indexercol_name] <= datetime.datetime.combine(startdate_first, datetime.datetime.min.time()))[0][-1]

    # Define dataframe range to be plotted
    df_plotted = dataframe.iloc[indStart:indEnd+1]

    if endyear is None :
        enddate = max(max(dataframe.iloc[indStart:indEnd+1].loc[colnames]))
        endyear = enddate.year
    # else:
    #     enddate = datetime.date(endyear,12,31)

    # Plot the studies on a timeline
    ax = plt.subplot()

    # Set plot colours
    colours_list = data_color(4,colormap)
    colours_list[:,-1] = 0.7

    # draw vertical lines every incremental year
    year_incr = [datetime.date( startyear_range[0] + year*increment, 1, 1)
                 for year in range(int( (endyear+1-startyear_range[0]) /increment ))]

    # Label axes and title
    ax.set_xlim(year_incr[0],year_incr[-1])
    ax.set_xticks(year_incr)
    ax.set_xticklabels([incr.year for incr in year_incr])
    ax.set_xlabel('Year')

    # ax.set_ylim([0, indEnd-indStart])
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(b=True, axis='both')

    for col in reversed(range(len(colnames))):
        ax.hlines(y=np.arange(len(df_plotted))[::-1],
                  xmin=df_plotted[indexercol_name],
                  xmax=df_plotted[colnames[col]],
                  colors=colours_list[col],linewidths=2)

    ax.legend(colnames)


mytimeline(study_dates_timestamp,
           startyear_range=[2019, 2021], endyear=None,
           title='Timeline of COVID-related studies',
           ylabel='Study Number')

### Interventional studies
- length of each study (also from CSV)
- what were some types of interventions used?
- in drug studies, what were the top 5 most popular ones studied?

### Any meaningful conclusions?
- In the small percentage of studies that actually have results posted, were any of them focused on COVID?
- If so, can we draw any meaningful conclusions?